In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/Ref-A'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/Ref-A


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+236.g6801d3d.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'Ref-A'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 2 # m
module_y = 1 # m. slope we will measure
sensorsy=2
torquetube = 0.15

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.5
hub_height = 1.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/Ref-A
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.500 avg
1 nonzero albedo values.


In [6]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y)


Module Name: PVmod
Module PVmod updated in module.json


In [7]:
cumulativesky = True
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [8]:
file = 'Phase2_meteo_hourly_psm3format.csv'
directory = os.path.expanduser('~/Documents/IEA-Bifacial-Tracking-Modeling')
weatherfile = os.path.join(directory, file)

metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-01-01_1000', endtime='2022-01-01_1200')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [9]:
metdata

{'latitude': 35.05, 'longitude': -106.54, 'elevation': 1600.0, 'timezone': -7.0, 'city': 'Albuquerque', 'datetime': [Timestamp('2022-01-01 10:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 11:30:00-0700', tz='Etc/GMT+7')], 'ghi': array([112, 265]), 'dhi': array([108, 232]), 'dni': array([ 5, 60]), 'albedo': None, 'dewpoint': None, 'pressure': None, 'temp_air': array([3.9, 3.3]), 'wind_speed': array([4.7, 5.5]), 'meastracker_angle': None, 'solpos':                            apparent_zenith     zenith  apparent_elevation  \
corrected_timestamp                                                         
2022-01-01 11:00:00-07:00        60.301624  60.325832           29.698376   
2022-01-01 12:00:00-07:00        58.030863  58.053009           31.969137   

                           elevation     azimuth  equation_of_time  
corrected_timestamp                                                 
2022-01-01 11:00:00-07:00  29.674168  161.465455         -3.652581  
2022-01-01 12:00:00-07:00  

In [10]:
trackerdict = demo.set1axis(**trackerParams)

Saving file EPWs/1axis_-30.0.csv, # points: 1
Saving file EPWs/1axis_-5.0.csv, # points: 1


In [11]:
if cumulativesky:
    demo.genCumSky1axis()
else:
    demo.gendaylit1axis()

message: There were 0 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-30.0.rad
message: There were 0 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies/1axis_-5.0.rad


In [12]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making .rad files for cumulativesky 1-axis workflow
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_-30.0.oct
Created 1axis_-5.0.oct


In [13]:
trakerdict = demo.analysis1axis(sensorsy=2)

Linescan in process: 1axis_-30.0_Row3_Module13_Front
Linescan in process: 1axis_-30.0_Row3_Module13_Back
Saved: results/irr_1axis_-30.0_Row3_Module13.csv
Index: -30.0. Wm2Front: 0.0. Wm2Back: 0.0
Linescan in process: 1axis_-5.0_Row3_Module13_Front
Linescan in process: 1axis_-5.0_Row3_Module13_Back
Saved: results/irr_1axis_-5.0_Row3_Module13.csv
Index: -5.0. Wm2Front: 0.0. Wm2Back: 0.0


In [14]:
trakerdict

{-30.0: {'csvfile': 'EPWs/1axis_-30.0.csv',
  'surf_azm': 90.0,
  'surf_tilt': 30.0,
  'datetime': Index(['2022-01-01 10:30:00'], dtype='object'),
  'count': 1,
  'skyfile': 'skies/1axis_-30.0.rad',
  'radfile': 'objects/1axis-30.0__C_1.25000_rtr_5.96000_tilt_30.00000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 2, 'y': 1, 'z': 0.02, 'modulematerial': 'black', 'scenex': 2.01, 'sceney': 1.0, 'scenez': 0.1, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 2 1 0.02 | xform -t -1.0 -0.5 0 -a 1 -t 0 1.0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.1}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.16778523489932887, 'text': '!xform -rx 30.0 -t 0 0 1.5 -a 25 -t 2.01 0 0 -a 5 -t 0 5.96 0 -i 1 -t -24.119999999999997 -11.92 0 -rz 90.0 -t 0 0 0 objects/PVmod.rad', 'radfiles': 'objects/1axis-30.0__C_1.25000_rtr_5.96000_tilt_30.00000_25modsx5rows_origin0,0.rad', 'sceneDict': {'tilt'